# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Question 1:  
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Since we are working with a NoSQL database, we must think about our query first. Looking at the question, we determine that our select statement will look something like:
SELECT Name of the artist, title of the song, length of the track FROM TABLE_NAME WHERE sessionId = value AND itemInSession = value

Consequently, we must create a table (if it doesn't exist) that will include the following information: 
- session id 
- item in session
- name of the artist
- song title
- song length

Therfore, we will create a table (CREATE TABLE IF NOT EXISTS) to store song details for by session id and item in session and use the fields session id and item in session as our primary keys since we need to filter on those two. : PRIMARY KEY(session_id, itemInSession).
Our column names will be:
- session_id 
- itemInSession
- artist_name
- song_title
- song_length


In [8]:
## First we build the table
query = "CREATE TABLE IF NOT EXISTS song_details_itemInSession "
query = query + "(session_id int, itemInSession int, artist_name text, song_title text, song_length float, PRIMARY KEY(session_id, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  INSERT statements for question1
        query = "INSERT INTO song_details_itemInSession(session_id, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## We run the SELECT statement corresponding to our first question
try:
    rows = session.execute("select session_id, itemInSession, artist_name, song_title, song_length from song_details_itemInSession where session_id = 338 and itemInSession = 4")
except Exception as e:
    print(e)

    
table = PrettyTable(['Artist', 'Song', 'Length'])

for row in rows:
    table.add_row([row.artist_name, row.song_title, row.song_length])    

print(table)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### Question 2:  
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Since we are working with a NoSQL database, we must think about our query first. Looking at the question, we determine that our select statement will look something like:
SELECT name of artist, song name and first name last name  FROM TABLE_NAME WHERE userid = value AND sessionid = value

Since our query contains a where that filters on user id and session id, these two fields will form our primary key. Furthermore, we must add item in session to the primary key in order to sort our output.

Consequently, we must create a table that will include the following information: 
- user id 
- session id
- item In Session (for sorting purposes)
- name of the artist
- song title
- first name
- last name


To ensure that sessions belonging to the same user end up on the same node, we use both userId and sessionId as partition keys so sessions from the same user are stored together to avoid the performance hit of having to access multiple nodes when the data set is large.
Therfore, we will create a table named song_playlist_session with the following primary key: PRIMARY KEY((userId, sessionId), itemInSession)

Our column names will be:
- userId
- sessionId
- itemInSession
- artist_name 
- song_name
- user_firstName
- user_lastName

In [11]:
## First we build the table
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userId int, sessionId int, itemInSession int, artist_name text, song_name text, user_firstName text, user_lastName text, \
                    PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)     

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session(userId, sessionId, itemInSession, artist_name, song_name, user_firstName, user_lastName)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        

In [13]:
## We test with a SELECT statement corresponding to our second question
try:
    rows = session.execute("select userId, sessionId, itemInSession, artist_name, song_name, user_firstName, user_lastName from song_playlist_session where userid = 10 and sessionid = 182")
except Exception as e:
    print(e)

table = PrettyTable(['Artist', 'Song', 'First Name', 'Last Name'])

for row in rows:
    table.add_row([row.artist_name, row.song_name, row.user_firstname, row.user_lastname])

print(table)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Question 3:  
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Since we are working with a NoSQL database, we must think about our query first. Looking at the question, we determine that our select statement will look something like:
SELECT user first name, user last name FROM TABLE_NAME WHERE song name = value

Our primary key must contain the song name. Furthermore, since we want to display list of unique users that played a given song, we are going to need the user id since the combination of first name and last name may not be unique. 

Consequently, we must create a table that will include the following information: 
- song name
- user id
- name of the artist
- user first name
- user last name

We then create a table to storethe list of unique user that listened to a specific song with the following primary key: PRIMARY KEY(song_title, userId))

Our column names will be:
- song_title
- userId
- artist_name
- user_firstName
- user_lastName

In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_list_by_song "
query = query + "(song_title text, userId int, artist_name text, user_firstName text, user_lastName text, PRIMARY KEY(song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_list_by_song(song_title, userId, artist_name, user_firstName, user_lastName)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[0], line[1], line[4]))

In [16]:
## We test using a SELECT statement corresponding to our question
try:
    rows = session.execute("select song_title, userId, artist_name, user_firstName, user_lastName \
                            from user_list_by_song where song_title = 'All Hands Against His Own'")
except Exception as e:
    print(e)

table = PrettyTable(['First Name', 'Last Name'])
    
for row in rows:
    table.add_row([row.user_firstname, row.user_lastname])
print(table)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
## We drop the table before closing out the sessions
try:
    session.execute("DROP TABLE song_details_itemInSession")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE song_playlist_session")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE user_list_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()